In [1]:
import numpy as np
import pandas as pd
from PIL import Image

import os
import logging

In [2]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s\n%(levelname)s:\n%(message)s")

# Preprocessing
### 순서
- ResNet50 파일에서는 단순히 224, 224의 이미지 크기 변환까지만 존재함
- 이미 배경이 지워진 이미지에서 시작했기 때문으로 추정함

In [3]:
root_path = os.getcwd()
static_path = os.path.join(root_path, "static")
root_path, static_path

('/Users/janghyolim/Desktop/code/HFoot',
 '/Users/janghyolim/Desktop/code/HFoot/static')

In [4]:
image_path = os.path.join(static_path, "Img", "78996949-c8f5-4538-82f5-52d5fade8854.jpg")
try:
    image = Image.open(image_path)
    
    if image.mode == "RGBA":
        image = image.convert("RGB")
    
    image_resized = image.resize((224,224))  #224, 224
    img_array = np.array(image_resized)/255.0#normalization
    
except Exception as e:
    print(f"Error Exception: {e}")

# 외부에서 모델가져와서 사용하기
### h5파일 가져와서 사용하기
- tf.keras.models.load_model([model_path])
- 그런데 지금 애러는 무슨 애러인지 잘 모르겠음
- 이건 나중에 물어봐서 해결해야할 것으로 보임

In [5]:
import tensorflow as tf

In [8]:
hand_seperate_model_path = os.path.join(static_path, "keras_model", "hand_seperate.h5")
try:
    model = tf.keras.models.load_model(hand_seperate_model_path)
    predictions = model.predict()
except Exception as e:
    logging.warning(f"Exception error: {e}")

2024-10-25 15:54:58,104
Exception error: Could not deserialize class 'Functional' because its parent module keras.src.engine.functional cannot be imported. Full object config: {'module': 'keras.src.engine.functional', 'class_name': 'Functional', 'config': {'name': 'model1', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 224, 224, 3], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_1'}, 'registered_name': None, 'name': 'input_1', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'ZeroPadding2D', 'config': {'name': 'Conv1_pad', 'trainable': True, 'dtype': 'float32', 'padding': [[0, 1], [0, 1]], 'data_format': 'channels_last'}, 'registered_name': None, 'build_config': {'input_shape': [None, 224, 224, 3]}, 'name': 'Conv1_pad', 'inbound_nodes': [[['input_1', 0, 0, {}]]]}, {'module': 'keras.layers', 'class_name': 'Conv2D', 'config': {'name': 'Conv1', 'trainable': True, 'dtype': 'flo